In [1]:
import numpy as np
import pandas as pd
import re
import tensorflow as tf

from gensim.models import KeyedVectors
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from os import path
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import LabelEncoder
from string import punctuation
from tensorflow.keras.layers import Concatenate, Conv1D, Dense, Embedding, GlobalMaxPooling1D, Input, LSTM, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm_notebook
from unidecode import unidecode

np.random.seed(42)
tf.compat.v1.random.set_random_seed(42)

# Data Ingestion

## Dataset constants

In [2]:
DIR_PATH = "../data/"
LANGUAGE = "spanish"
DROP_COLUMNS = ["split", "language"]
UNRELIABLE_SAMPLING = 0.25

## Dataset Loading

In [ ]:
%%time
def load_data(base_path, language, drop_columns, unreliable_sampling):
    datasets = {}
    for ds in tqdm_notebook(["train_reliable", "train_unreliable", "dev", "test"]):
        if ds == "train_unreliable" and unreliable_sampling == 0:
            continue
        
        df = pd.read_parquet(
            path.join(base_path, f"{language}", f"{ds}.parquet")
        ).drop(drop_columns, axis=1, errors="ignore")
        
        if ds == "train_unreliable" and 0 < unreliable_sampling < 1:
            df = df.groupby(["category"]).apply(
                lambda cat: cat.sample(frac=unreliable_sampling)
            ).reset_index(drop=True)
        elif ds == "train_unreliable" and unreliable_sampling > 1:
            df = df.groupby(["category"]).apply(
                lambda cat: cat.sample(n=int(unreliable_sampling))
            ).reset_index(drop=True)
        
        if ds == "train_reliable":
            datasets["train"] = df
        elif ds == "train_unreliable":
            datasets["train"] = pd.concat([
                datasets["train"],
                df
            ], ignore_index=True)
        else:
            datasets[ds] = df
    
    w2v = KeyedVectors.load_word2vec_format(
        path.join(base_path, f"{language}", "word2vec.bin.gz"), 
        binary=True
    )
    
    return datasets, w2v

datasets, w2v = load_data(DIR_PATH, LANGUAGE, DROP_COLUMNS, UNRELIABLE_SAMPLING)

# Data Preprocessing

## Label Encoding

In [ ]:
%%time
def label_encoder(*dfs):
    labels = pd.concat(dfs)["category"].tolist()
    lbl_enc = LabelEncoder().fit(labels)

    return lbl_enc

lbl_enc = label_encoder(datasets["train"], datasets["dev"])

for split in ["train", "dev"]:
    datasets[split]["target"] = lbl_enc.transform(datasets[split]["category"])
    datasets[split].drop(["category"], axis=1, inplace=True)

## Text curation

### Capitalization

In [ ]:
%%time

def lowercase_titles(datasets, column="title"):
    for split in tqdm_notebook(datasets):
        datasets[split]["lowercase_title"] = datasets[split][column].str.lower()
    return datasets

datasets = lowercase_titles(datasets)

### Tokenization

In [ ]:
%%time

def tokenization(datasets, language, column="title"):
    for split in tqdm_notebook(datasets):
        datasets[split]["tokens"] = datasets[split][column].apply(
            lambda title: word_tokenize(title, language=language)
        )
    return datasets

datasets = tokenization(datasets, LANGUAGE, "lowercase_title")

### Punctuation removal

In [ ]:
%%time

def remove_punctuation(datasets, punctuation, column="tokens"):
    for split in tqdm_notebook(datasets):
        datasets[split]["non_punct_tokens"] = datasets[split][column].apply(
            lambda words: [w for w in words if w not in punctuation]
        )
    return datasets

datasets = remove_punctuation(datasets, punctuation, "words")

### Stopwords removal

In [ ]:
%%time

def remove_stopwords(datasets, stopwords, column="tokens"):
    for split in tqdm_notebook(datasets):
        datasets[split]["non_sw_tokens"] = datasets[split][column].apply(
            lambda words: [w for w in words if w not in stopwords]
        )
    return datasets

datasets = remove_stopwords(datasets, set(stopwords.words(LANGUAGE)), "non_punct_tokens")

### Word Vectorization

In [ ]:
%%time

def word_with_vector(word, w2v, stemmer):
    if word in w2v:
        return word
    elif word.capitalize() in w2v:
        return word.capitalize()
    elif unidecode(word) in w2v:
        return unidecode(word)
    elif unidecode(word.capitalize()) in w2v:
        return unidecode(word.capitalize())
    elif stemmer.stem(word) in w2v:
        return stemmer.stem(word)
    elif word.isdigit():
        return "DIGITO"
    else:
        return "<UNK>"
    # TODO: Lemmatization? Other normalizations?

def word_vectorize(datasets, language, w2v, column="tokens"):
    stemmer = SnowballStemmer(language)
    for split in tqdm_notebook(datasets):
        datasets[split]["normalized_title"] = datasets[split][column].apply(
            lambda words: [word_with_vector(w, w2v, stemmer) for w in words]
        )
    return datasets

datasets = word_vectorize(datasets, LANGUAGE, w2v, "non_punct_tokens")

In [ ]:
%%time
def words_to_idx(all_words, w2v, null_token="<NULL>",
                 unknown_token="<UNK>", num_token="DIGITO"):
    word_index = {word for words in all_words for word in words if word in w2v}
    word_index = {word: idx for idx, word in enumerate(sorted(word_index), start=1)}
    word_index[null_token] = 0
    if num_token not in word_index:
        word_index[num_token] = len(word_index)
    word_index[unknown_token] = len(word_index)

    return word_index

word_index = words_to_idx(pd.concat(list(datasets.values()), sort=False)["normalized_title"], w2v)

print(f"Vocab length: {len(word_index)}")

In [ ]:
%%time

MAX_SEQUENCE_LEN = 15

def sequence_padding(series, word_index, max_len):
    return pad_sequences(
            series.apply(
                lambda words: [word_index.get(word, word_index["<UNK>"]) for word in words]
            ).tolist(), maxlen=max_len
        )

train_word_sequences = sequence_padding(
    datasets["train"]["normalized_title"], word_index, MAX_SEQUENCE_LEN
)

dev_word_sequences = sequence_padding(
    datasets["dev"]["normalized_title"], word_index, MAX_SEQUENCE_LEN
)

test_word_sequences = sequence_padding(
    datasets["test"]["normalized_title"], word_index, MAX_SEQUENCE_LEN
)

In [ ]:
%%time

train_target = to_categorical(
    datasets["train"]["target"].tolist(),
    num_classes=lbl_enc.classes_.shape[0]
)

dev_target = to_categorical(
    datasets["dev"]["target"].tolist(),
    num_classes=lbl_enc.classes_.shape[0]
)

In [ ]:
%%time
def get_embedding_matrix(word_index, w2v):
    embedding_matrix = np.zeros((len(word_index), w2v.vector_size))

    for word, i in word_index.items():
        if word in w2v and word not in {"<NULL>", "<UNK>", "<NUM>"}:
            embedding_matrix[i] = w2v[word]
        elif word == "<UNK>" or word == "<NUM>":
            embedding_matrix[i] = np.random.normal(size=(w2v.vector_size,))

    return embedding_matrix

word_embedding_matrix = get_embedding_matrix(word_index, w2v)

# CNN Building

## Model Constants

In [ ]:
FILTERS = [2, 3, 4, 5]
FILTER_COUNT = 128
ACTIVATION = "relu"
PADDING = "same"

## Model Building

In [ ]:
def build_model(word_vocab_size, word_vector_size, word_embedding_matrix, output_size, max_sequence_len,
                filters, filter_count, activation="relu", padding="valid"):
    word_embedding_layer = Embedding(word_vocab_size, word_vector_size,
                                     weights=[word_embedding_matrix],
                                     input_length=max_sequence_len,
                                     trainable=False)

    word_sequence_input = Input(shape=(max_sequence_len,))
    word_embedded_sequences = word_embedding_layer(word_sequence_input)

    layers = []
    for filter_size in filters:
        layer = Conv1D(
            filter_count,
            filter_size,
            activation=activation,
            padding=padding
        )(word_embedded_sequences)
        layers.append(GlobalMaxPooling1D()(layer))

    layer = Concatenate()(layers)
    preds = Dense(output_size, activation="softmax")(layer)
    model = Model(word_sequence_input, preds)

    return model

model = build_model(
    word_vocab_size=len(word_index),
    word_vector_size=w2v.vector_size, 
    word_embedding_matrix=word_embedding_matrix,
    output_size=lbl_enc.classes_.shape[0],
    max_sequence_len=MAX_SEQUENCE_LEN,
    filters=FILTERS,
    filter_count=FILTER_COUNT,
    activation=ACTIVATION,
    padding=PADDING
)

model.compile(
    optimizer="nadam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# Fitting the CNN

In [ ]:
model.fit(
    x=train_word_sequences,
    y=train_target,
    batch_size=4096,
    epochs=5,
    validation_data=(dev_word_sequences, dev_target),
    validation_freq=1
)

In [ ]:
datasets["dev"]["predictions"] = model.predict(
    dev_word_sequences, batch_size=1024, verbose=0
).argmax(axis=1)

In [ ]:
balanced_accuracy_score(datasets["dev"]["target"], datasets["dev"]["predictions"])